In [1]:
def set_up_shear_test(E,nu,c,phi,psi,sig_n,confined,t_discrete):
    m = mtest.MTest()
    mtest.setVerboseMode(mtest.VerboseLevel.VERBOSE_QUIET)
    m.setMaximumNumberOfSubSteps(1)
    m.setBehaviour('generic', './src/libBehaviour.so', 'MohrCoulombAbboSloan')
    m.setExternalStateVariable("Temperature", 293.15)
    
    m.setImposedStress('SXX', {0: 0, 1: sig_n})
    if confined:
        m.setImposedStrain('EYY', 0.0)
        m.setImposedStrain('EZZ', 0.0)
    else:
        m.setImposedStress('SYY',{0: 0, 1: sig_n*(1-np.sin(np.deg2rad(phi)))})
        m.setImposedStress('SZZ', {0: 0, 1: sig_n*(1-np.sin(np.deg2rad(phi)))})
    m.setImposedStress('SXZ', 0.0)
    m.setImposedStress('SYZ', 0.0)
    m.setImposedStrain('EXY', {0: 0, 1: 0, 2: -np.sqrt(2)*2e-2}) #Kelvin Mapping of shear components!!
    
    m.setMaterialProperty('YoungModulus', E)
    m.setMaterialProperty('PoissonRatio', nu)
    m.setMaterialProperty('Cohesion', c)
    m.setMaterialProperty('FrictionAngle', phi)
    m.setMaterialProperty('DilatancyAngle', psi)
    m.setMaterialProperty('TransitionAngle', 29)
    m.setMaterialProperty('TensionCutOffParameter', 0)

    s = mtest.MTestCurrentState()
    wk = mtest.MTestWorkSpace()
    m.completeInitialisation()
    m.initializeCurrentState(s)
    m.initializeWorkSpace(wk)
    
    
    tau_xy = np.array([0.])
    eps_xy = np.array([0.])
    tau_yz = np.array([0.])
    eps_yz = np.array([0.])
    tau_xz = np.array([0.])
    eps_xz = np.array([0.])
    eps_xx = np.array([0.])
    sig_xx = np.array([0.])
    eps_yy = np.array([0.])
    sig_yy = np.array([0.])
    eps_zz = np.array([0.])
    sig_zz = np.array([0.])
    for i in range(len(t_discrete)-1):
        m.execute(s, wk, t_discrete[i], t_discrete[i + 1])
        eps_xy = np.append(eps_xy,s.e1[3]/np.sqrt(2.)) #Kelvin mapping backwards!
        tau_xy = np.append(tau_xy,s.s1[3]/np.sqrt(2.)) #Kelvin mapping backwards!
        eps_xz = np.append(eps_xz,s.e1[4]/np.sqrt(2.)) #Kelvin mapping backwards!
        tau_xz = np.append(tau_xz,s.s1[4]/np.sqrt(2.)) #Kelvin mapping backwards!
        eps_yz = np.append(eps_yz,s.e1[5]/np.sqrt(2.)) #Kelvin mapping backwards!
        tau_yz = np.append(tau_yz,s.s1[5]/np.sqrt(2.)) #Kelvin mapping backwards!
        sig_xx = np.append(sig_xx,s.s1[0])
        eps_xx = np.append(eps_xx,s.e1[0])
        sig_yy = np.append(sig_yy,s.s1[1])
        eps_yy = np.append(eps_yy,s.e1[1])
        sig_zz = np.append(sig_zz,s.s1[2])
        eps_zz = np.append(eps_zz,s.e1[2])

    return -sig_xx, -eps_xx, -sig_yy, -eps_yy, -sig_zz, -eps_zz, -tau_xy, -eps_xy, -tau_xz, -eps_xz, -tau_yz, -eps_yz

In [2]:
def compute_q(sxx,syy,szz,sxy,syz,sxz):
    q = np.array([])
    for i in range(len(sig_xx)):
        s_tens = np.array([[sxx.T[i],sxy.T[i],sxz.T[i]],
                           [sxy.T[i],syy.T[i],syz.T[i]],
                           [sxz.T[i],syz.T[i],szz.T[i]]])
        s_dev = s_tens - np.trace(s_tens)/3 * np.eye(3)
        q = np.append(q,np.sqrt(np.tensordot(s_dev,s_dev,2)*3/2))
    return q

In [3]:
def compute_lode(sxx,syy,szz,sxy,syz,sxz):
    lode = np.array([0])
    for i in range(1,len(sig_xx)):
        s_tens = np.array([[sxx.T[i],sxy.T[i],sxz.T[i]],
                           [sxy.T[i],syy.T[i],syz.T[i]],
                           [sxz.T[i],syz.T[i],szz.T[i]]])
        s_dev = s_tens - np.trace(s_tens)/3 * np.eye(3)
        qi = np.sqrt(np.tensordot(s_dev,s_dev,2)*3/2)
        det_s_dev = np.linalg.det(s_dev)
        if qi > 1e-5:
            arg = det_s_dev/qi**3 * 27./2.
            if arg < -1:
                arg = -1
            if arg > 1:
                arg = 1
            lode = np.append(lode,np.arccos(arg)/3)
        else:
            lode = np.append(lode,0.)
    return lode

In [4]:
def compute_princ(sxx,syy,szz,sxy,syz,sxz):
    s1 = np.array([])
    s3 = np.array([])
    for i in range(len(sig_xx)):
        s_tens = np.array([[sxx.T[i],sxy.T[i],sxz.T[i]],
                           [sxy.T[i],syy.T[i],syz.T[i]],
                           [sxz.T[i],syz.T[i],szz.T[i]]])
        evals = np.linalg.eigvals(s_tens)
        s1 = np.append(s1,np.max(evals))
        s3 = np.append(s3,np.min(evals))
    return s1, s3

In [5]:
def plot_results(sig_xx, eps_xx, sig_yy, eps_yy, sig_zz, eps_zz, tau_xy, eps_xy, tau_xz, eps_xz, tau_yz, eps_yz, figlabel):
    ev = (eps_xx + eps_yy + eps_zz)
    p = (sig_xx + sig_yy + sig_zz)/3
    q = compute_q(sig_xx,sig_yy,sig_zz,tau_xy,tau_yz,tau_xz)
    theta = compute_lode(sig_xx,sig_yy,sig_zz,tau_xy,tau_yz,tau_xz)
    s1, s3 = compute_princ(sig_xx,sig_yy,sig_zz,tau_xy,tau_yz,tau_xz)
    fig,ax = plt.subplots(ncols=2,nrows=3,figsize=(18,18))
    ax[0][0].plot(times,eps_xx*1e2,marker='d',label='$\\epsilon_{xx}$')
    ax[0][0].plot(times,eps_yy*1e2,marker='d',label='$\\epsilon_{yy}$')
    ax[0][0].plot(times,eps_zz*1e2,marker='d',label='$\\epsilon_{zz}$')
    ax[0][0].plot(times,eps_xy*1e2,marker='d',label='$\\epsilon_{xy}$')
    ax[0][0].plot(times,eps_yz*1e2,marker='d',label='$\\epsilon_{yz}$')
    ax[0][0].plot(times,eps_xz*1e2,marker='d',label='$\\epsilon_{xz}$')
    ax[0][0].plot(times,ev*1e2,marker='d',label='$\\epsilon_{vol}$',color='black')
    ax[0][0].set_xlabel('$t$ / d')
    ax[0][0].set_ylabel('$\\epsilon$ / %')
    ax[0][0].legend()
    #
    ax[0][1].plot(times,sig_xx/1e3,marker='d',label='$\\sigma_{xx}$')
    ax[0][1].plot(times,sig_yy/1e3,marker='d',label='$\\sigma_{yy}$')
    ax[0][1].plot(times,sig_zz/1e3,marker='d',label='$\\sigma_{zz}$')
    ax[0][1].plot(times,tau_xy/1e3,marker='d',label='$\\sigma_{xy}$')
    ax[0][1].plot(times,tau_yz/1e3,marker='d',label='$\\sigma_{yz}$')
    ax[0][1].plot(times,tau_xz/1e3,marker='d',label='$\\sigma_{xz}$')
    ax[0][1].set_xlabel('$t$ / d')
    ax[0][1].set_ylabel('$\\sigma$ / kPa')
    ax[0][1].legend()
    #
    ax[1][0].plot(p/1e3,q/1e3,marker='d')
    ax[1][0].set_xlabel('$p$ / kPa')
    ax[1][0].set_ylabel('$q$ / kPa')
    #
    ax[1][1].plot(eps_xx*100.,sig_xx/1e3,marker='d',label='$\\sigma_{xx}$, $\\epsilon_{xx}$')
    ax[1][1].plot(eps_yy*100.,sig_yy/1e3,marker='d',label='$\\sigma_{yy}$, $\\epsilon_{yy}$')
    ax[1][1].plot(eps_zz*100.,sig_zz/1e3,marker='d',label='$\\sigma_{zz}$, $\\epsilon_{zz}$')
    ax[1][1].plot(eps_xy*100.,tau_xy/1e3,marker='d',label='$\\sigma_{xy}$, $\\epsilon_{xy}$',ls='--')
    ax[1][1].plot(eps_yz*100.,tau_yz/1e3,marker='d',label='$\\sigma_{yz}$, $\\epsilon_{yz}$',ls='--')
    ax[1][1].plot(eps_xz*100.,tau_xz/1e3,marker='d',label='$\\sigma_{xz}$, $\\epsilon_{xz}$',ls='--')
    ax[1][1].plot(ev*100.,p/1e3,marker='d',label='$p$, $\\epsilon_\\mathrm{vol}$',color='black')
    ax[1][1].set_xlabel('$\\epsilon$ / %')
    ax[1][1].set_ylabel('$\\sigma$ / kPa')
    ax[1][1].legend()
    #
    ax[2][0].plot(times,np.rad2deg(theta),label='current test',marker='d')
    ax[2][0].axhline(np.rad2deg(0),label='triaxial compression',ls='--')
    ax[2][0].axhline(np.rad2deg(np.pi/3),label='triaxial extension',ls=':')
    ax[2][0].axhline(np.rad2deg(np.pi/6),label='simple shear',ls='-.')
    ax[2][0].set_xlabel('$t$ / d')
    ax[2][0].set_ylabel('$\\theta_c$ / °')
    ax[2][0].legend()
    #
    MC_line = lambda sm: sm * np.sin(np.deg2rad(phi)) + c * np.cos(np.deg2rad(phi))
    sig_mean = (s1+s3)/2
    tau_max = (s1-s3)/2
    ax[2][1].plot(sig_mean/1e3,tau_max/1e3,marker='d',label='stress path')
    ax[2][1].plot(sig_mean/1e3,MC_line(sig_mean)/1e3,ls='--',color='black',label='$\\frac{\\sigma_1 - \\sigma_3}{2} = \\frac{\\sigma_1 + \\sigma_3}{2} \\sin \\varphi + c \\cos\\varphi$')
    ax[2][1].set_xlabel('$\\frac{\\sigma_1 + \\sigma_3}{2}$ / kPa')
    ax[2][1].set_ylabel('$\\frac{\\sigma_1 - \\sigma_3}{2}$ / kPa')
    ax[2][1].legend()
    #
    fig.suptitle(figlabel)
    fig.tight_layout();

In [6]:
def set_up_triax_test(E,nu,c,phi,psi,p_con,e_ax,t_discrete):
    m = mtest.MTest()
    mtest.setVerboseMode(mtest.VerboseLevel.VERBOSE_QUIET)
    m.setMaximumNumberOfSubSteps(1)
    #m.setModellingHypothesis("Axisymmetrical")
    m.setBehaviour('generic', './src/libBehaviour.so', 'MohrCoulombAbboSloan')
    m.setExternalStateVariable("Temperature", 293.15)
    
    e_con = p_con * (1 - 2 * nu) / E
    m.setImposedStress("SXX", {0: 0, 1: -p_con, 2: -p_con})
    m.setImposedStress("SYY", {0: 0, 1: -p_con, 2: -p_con})
    m.setImposedStrain('EZZ', {0: 0, 1: -e_con, 2: -e_ax})
    
    m.setMaterialProperty('YoungModulus', E)
    m.setMaterialProperty('PoissonRatio', nu)
    m.setMaterialProperty('Cohesion', c)
    m.setMaterialProperty('FrictionAngle', phi)
    m.setMaterialProperty('DilatancyAngle', psi)
    m.setMaterialProperty('TransitionAngle', 29)
    m.setMaterialProperty('TensionCutOffParameter', 0)

    s = mtest.MTestCurrentState()
    wk = mtest.MTestWorkSpace()
    m.completeInitialisation()
    m.initializeCurrentState(s)
    m.initializeWorkSpace(wk)
    
    
    tau_xy = np.array([0.])
    eps_xy = np.array([0.])
    tau_yz = np.array([0.])
    eps_yz = np.array([0.])
    tau_xz = np.array([0.])
    eps_xz = np.array([0.])
    eps_xx = np.array([0.])
    sig_xx = np.array([0.])
    eps_yy = np.array([0.])
    sig_yy = np.array([0.])
    eps_zz = np.array([0.])
    sig_zz = np.array([0.])
    for i in range(len(t_discrete)-1):
        m.execute(s, wk, t_discrete[i], t_discrete[i + 1])
        eps_xy = np.append(eps_xy,s.e1[3]/np.sqrt(2.)) #Kelvin mapping backwards!
        tau_xy = np.append(tau_xy,s.s1[3]/np.sqrt(2.)) #Kelvin mapping backwards!
        eps_xz = np.append(eps_xz,s.e1[4]/np.sqrt(2.)) #Kelvin mapping backwards!
        tau_xz = np.append(tau_xz,s.s1[4]/np.sqrt(2.)) #Kelvin mapping backwards!
        eps_yz = np.append(eps_yz,s.e1[5]/np.sqrt(2.)) #Kelvin mapping backwards!
        tau_yz = np.append(tau_yz,s.s1[5]/np.sqrt(2.)) #Kelvin mapping backwards!
        sig_xx = np.append(sig_xx,s.s1[0])
        eps_xx = np.append(eps_xx,s.e1[0])
        sig_yy = np.append(sig_yy,s.s1[1])
        eps_yy = np.append(eps_yy,s.e1[1])
        sig_zz = np.append(sig_zz,s.s1[2])
        eps_zz = np.append(eps_zz,s.e1[2])

    return -sig_xx, -eps_xx, -sig_yy, -eps_yy, -sig_zz, -eps_zz, -tau_xy, -eps_xy, -tau_xz, -eps_xz, -tau_yz, -eps_yz

In [7]:
def set_up_triax_test_ext(E,nu,c,phi,psi,p_con,t_discrete):
    m = mtest.MTest()
    mtest.setVerboseMode(mtest.VerboseLevel.VERBOSE_QUIET)
    m.setMaximumNumberOfSubSteps(10)
    #m.setModellingHypothesis("Axisymmetrical")
    m.setBehaviour('generic', './src/libBehaviour.so', 'MohrCoulombAbboSloan')
    m.setExternalStateVariable("Temperature", 293.15)
    
    e_con = p_con * (1 - 2 * nu) / E
    m.setImposedStress("SXX", {0: 0, 1: -p_con, 2: -p_con})
    m.setImposedStress("SYY", {0: 0, 1: -p_con, 2: -p_con})
    m.setImposedStrain('EZZ', {0: 0, 1: -e_con, 2: e_con})
    
    m.setMaterialProperty('YoungModulus', E)
    m.setMaterialProperty('PoissonRatio', nu)
    m.setMaterialProperty('Cohesion', c)
    m.setMaterialProperty('FrictionAngle', phi)
    m.setMaterialProperty('DilatancyAngle', psi)
    m.setMaterialProperty('TransitionAngle', 29)
    m.setMaterialProperty('TensionCutOffParameter', 0)

    s = mtest.MTestCurrentState()
    wk = mtest.MTestWorkSpace()
    m.completeInitialisation()
    m.initializeCurrentState(s)
    m.initializeWorkSpace(wk)
    
    
    tau_xy = np.array([0.])
    eps_xy = np.array([0.])
    tau_yz = np.array([0.])
    eps_yz = np.array([0.])
    tau_xz = np.array([0.])
    eps_xz = np.array([0.])
    eps_xx = np.array([0.])
    sig_xx = np.array([0.])
    eps_yy = np.array([0.])
    sig_yy = np.array([0.])
    eps_zz = np.array([0.])
    sig_zz = np.array([0.])
    for i in range(len(t_discrete)-1):
        m.execute(s, wk, t_discrete[i], t_discrete[i + 1])
        eps_xy = np.append(eps_xy,s.e1[3]/np.sqrt(2.)) #Kelvin mapping backwards!
        tau_xy = np.append(tau_xy,s.s1[3]/np.sqrt(2.)) #Kelvin mapping backwards!
        eps_xz = np.append(eps_xz,s.e1[4]/np.sqrt(2.)) #Kelvin mapping backwards!
        tau_xz = np.append(tau_xz,s.s1[4]/np.sqrt(2.)) #Kelvin mapping backwards!
        eps_yz = np.append(eps_yz,s.e1[5]/np.sqrt(2.)) #Kelvin mapping backwards!
        tau_yz = np.append(tau_yz,s.s1[5]/np.sqrt(2.)) #Kelvin mapping backwards!
        sig_xx = np.append(sig_xx,s.s1[0])
        eps_xx = np.append(eps_xx,s.e1[0])
        sig_yy = np.append(sig_yy,s.s1[1])
        eps_yy = np.append(eps_yy,s.e1[1])
        sig_zz = np.append(sig_zz,s.s1[2])
        eps_zz = np.append(eps_zz,s.e1[2])

    return -sig_xx, -eps_xx, -sig_yy, -eps_yy, -sig_zz, -eps_zz, -tau_xy, -eps_xy, -tau_xz, -eps_xz, -tau_yz, -eps_yz

In [8]:
def MCC_triax_test(E,nu,la,ka,M,v0,phi0,pc0,pamb,p_con,e_ax,t_discrete):
    m = mtest.MTest()
    mtest.setVerboseMode(mtest.VerboseLevel.VERBOSE_QUIET)
    m.setMaximumNumberOfSubSteps(10)
    #m.setModellingHypothesis("Axisymmetrical")
    m.setBehaviour('generic', './src/libBehaviour.so', 'ModCamClayStandard')
    m.setExternalStateVariable("Temperature", 293.15)
    
    m.setStress([-p_con , -p_con, -p_con, 0.0, 0.0, 0.0])
    m.setStrain([0.0, 0.0, 0.0 , 0.0, 0.0, 0.0])
    #e_con = p_con * (1 - 2 * nu) / E
    m.setImposedStress("SXX", {0: -p_con, 1: -p_con})
    m.setImposedStress("SYY", {0: -p_con, 1: -p_con})
    m.setImposedStrain('EZZ', {0: 0, 1: -e_ax})

    m.setMaterialProperty("PoissonRatio", nu)
    m.setMaterialProperty("CriticalStateLineSlope", M)
    m.setMaterialProperty("SwellingLineSlope", ka)
    m.setMaterialProperty("VirginConsolidationLineSlope", la)
    m.setMaterialProperty("InitialPreConsolidationPressure", -pc0)
    m.setMaterialProperty("InitialVoidRatio", 1/(1-phi0))

    s = mtest.MTestCurrentState()
    wk = mtest.MTestWorkSpace()
    m.completeInitialisation()
    m.initializeCurrentState(s)
    m.initializeWorkSpace(wk)
    
    
    tau_xy = np.array([s.s0[3]/np.sqrt(2.)])
    eps_xy = np.array([s.e0[3]/np.sqrt(2.)])
    tau_yz = np.array([s.s0[5]/np.sqrt(2.)])
    eps_yz = np.array([s.e0[5]/np.sqrt(2.)])
    tau_xz = np.array([s.s0[4]/np.sqrt(2.)])
    eps_xz = np.array([s.e0[4]/np.sqrt(2.)])
    eps_xx = np.array([s.e0[0]])
    sig_xx = np.array([s.s0[0]])
    eps_yy = np.array([s.e0[1]])
    sig_yy = np.array([s.s0[1]])
    eps_zz = np.array([s.e0[2]])
    sig_zz = np.array([s.s0[2]])
    pc = np.array([-pc0])
    for i in range(len(t_discrete)-1):
        m.execute(s, wk, t_discrete[i], t_discrete[i + 1])
        eps_xy = np.append(eps_xy,s.e1[3]/np.sqrt(2.)) #Kelvin mapping backwards!
        tau_xy = np.append(tau_xy,s.s1[3]/np.sqrt(2.)) #Kelvin mapping backwards!
        eps_xz = np.append(eps_xz,s.e1[4]/np.sqrt(2.)) #Kelvin mapping backwards!
        tau_xz = np.append(tau_xz,s.s1[4]/np.sqrt(2.)) #Kelvin mapping backwards!
        eps_yz = np.append(eps_yz,s.e1[5]/np.sqrt(2.)) #Kelvin mapping backwards!
        tau_yz = np.append(tau_yz,s.s1[5]/np.sqrt(2.)) #Kelvin mapping backwards!
        sig_xx = np.append(sig_xx,s.s1[0])
        eps_xx = np.append(eps_xx,s.e1[0])
        sig_yy = np.append(sig_yy,s.s1[1])
        eps_yy = np.append(eps_yy,s.e1[1])
        sig_zz = np.append(sig_zz,s.s1[2])
        eps_zz = np.append(eps_zz,s.e1[2])
        pc = np.append(pc,s.getInternalStateVariableValue("PreConsolidationPressure"))

    return -sig_xx, -eps_xx, -sig_yy, -eps_yy, -sig_zz, -eps_zz, -tau_xy, -eps_xy, -tau_xz, -eps_xz, -tau_yz, -eps_yz, -pc

In [9]:
def plot_results_MCC(sig_xx, eps_xx, sig_yy, eps_yy, sig_zz, eps_zz, tau_xy, eps_xy, tau_xz, eps_xz, tau_yz, eps_yz, pc,M, figlabel):
    ev = (eps_xx + eps_yy + eps_zz)
    p = (sig_xx + sig_yy + sig_zz)/3
    q = compute_q(sig_xx,sig_yy,sig_zz,tau_xy,tau_yz,tau_xz)
    theta = compute_lode(sig_xx,sig_yy,sig_zz,tau_xy,tau_yz,tau_xz)
    s1, s3 = compute_princ(sig_xx,sig_yy,sig_zz,tau_xy,tau_yz,tau_xz)
    fig,ax = plt.subplots(ncols=2,nrows=2,figsize=(18,12))
    ax[0][0].plot(times,eps_xx*1e2,marker='d',label='$\\epsilon_{xx}$')
    ax[0][0].plot(times,eps_yy*1e2,marker='d',label='$\\epsilon_{yy}$')
    ax[0][0].plot(times,eps_zz*1e2,marker='d',label='$\\epsilon_{zz}$')
    ax[0][0].plot(times,eps_xy*1e2,marker='d',label='$\\epsilon_{xy}$')
    ax[0][0].plot(times,eps_yz*1e2,marker='d',label='$\\epsilon_{yz}$')
    ax[0][0].plot(times,eps_xz*1e2,marker='d',label='$\\epsilon_{xz}$')
    ax[0][0].plot(times,ev*1e2,marker='d',label='$\\epsilon_{vol}$',color='black')
    ax[0][0].set_xlabel('$t$ / d')
    ax[0][0].set_ylabel('$\\epsilon$ / %')
    ax[0][0].legend()
    #
    ax[0][1].plot(times,sig_xx/1e3,marker='d',label='$\\sigma_{xx}$')
    ax[0][1].plot(times,sig_yy/1e3,marker='d',label='$\\sigma_{yy}$')
    ax[0][1].plot(times,sig_zz/1e3,marker='d',label='$\\sigma_{zz}$')
    ax[0][1].plot(times,tau_xy/1e3,marker='d',label='$\\sigma_{xy}$')
    ax[0][1].plot(times,tau_yz/1e3,marker='d',label='$\\sigma_{yz}$')
    ax[0][1].plot(times,tau_xz/1e3,marker='d',label='$\\sigma_{xz}$')
    ax[0][1].set_xlabel('$t$ / d')
    ax[0][1].set_ylabel('$\\sigma$ / kPa')
    ax[0][1].legend()
    #
    qy = lambda p, pc, M: np.sqrt(M*M * (p * (pc - p)))
    CSL = lambda p,M: M*p
    px = np.linspace(0,pc[0],1000)
    ax[1][0].plot(p/1e3,q/1e3,marker='d',label='stress path')
    ax[1][0].plot(px/1e3,qy(px,pc[0],M)/1e3,label='initial yield surface')
    ax[1][0].plot(px/1e3,CSL(px,M)/1e3,label='CSL')
    px = np.linspace(0,pc[-1],1000)
    ax[1][0].plot(px/1e3,qy(px,pc[-1],M)/1e3,label='final yield surface')
    ax[1][0].set_xlabel('$p$ / kPa')
    ax[1][0].set_ylabel('$q$ / kPa')
    ax[1][0].legend()
    #
    ax[1][1].plot(eps_xx*100.,sig_xx/1e3,marker='d',label='$\\sigma_{xx}$, $\\epsilon_{xx}$')
    ax[1][1].plot(eps_yy*100.,sig_yy/1e3,marker='d',label='$\\sigma_{yy}$, $\\epsilon_{yy}$')
    ax[1][1].plot(eps_zz*100.,sig_zz/1e3,marker='d',label='$\\sigma_{zz}$, $\\epsilon_{zz}$')
    ax[1][1].plot(eps_xy*100.,tau_xy/1e3,marker='d',label='$\\sigma_{xy}$, $\\epsilon_{xy}$',ls='--')
    ax[1][1].plot(eps_yz*100.,tau_yz/1e3,marker='d',label='$\\sigma_{yz}$, $\\epsilon_{yz}$',ls='--')
    ax[1][1].plot(eps_xz*100.,tau_xz/1e3,marker='d',label='$\\sigma_{xz}$, $\\epsilon_{xz}$',ls='--')
    ax[1][1].plot(ev*100.,p/1e3,marker='d',label='$p$, $\\epsilon_\\mathrm{vol}$',color='black')
    ax[1][1].set_xlabel('$\\epsilon$ / %')
    ax[1][1].set_ylabel('$\\sigma$ / kPa')
    ax[1][1].legend()
    #
    fig.suptitle(figlabel)
    fig.tight_layout();

In [10]:
def MCC_triax_test_ext(E,nu,la,ka,M,v0,phi0,pc0,pamb,p_con,t_discrete):
    m = mtest.MTest()
    mtest.setVerboseMode(mtest.VerboseLevel.VERBOSE_QUIET)
    m.setMaximumNumberOfSubSteps(10)
    #m.setModellingHypothesis("Axisymmetrical")
    m.setBehaviour('generic', './src/libBehaviour.so', 'ModCamClayStandard')
    m.setExternalStateVariable("Temperature", 293.15)
    
    e_con = p_con * (1 - 2 * nu) / E
    m.setStress([-p_con , -p_con, -p_con, 0.0, 0.0, 0.0]) 
    m.setStrain([0.0, 0.0, 0.0 , 0.0, 0.0, 0.0])
    m.setImposedStress("SXX", {0: -p_con, 1: -p_con})
    m.setImposedStress("SYY", {0: -p_con, 1: -p_con})
    m.setImposedStrain('EZZ', {0: 0, 1: 1.2*e_con})
    
    m.setMaterialProperty("PoissonRatio", nu)
    m.setMaterialProperty("CriticalStateLineSlope", M)
    m.setMaterialProperty("SwellingLineSlope", ka)
    m.setMaterialProperty("VirginConsolidationLineSlope", la)
    m.setMaterialProperty("InitialPreConsolidationPressure", -pc0)
    m.setMaterialProperty("InitialVoidRatio", 1/(1-phi0))

    s = mtest.MTestCurrentState()
    wk = mtest.MTestWorkSpace()
    m.completeInitialisation()
    m.initializeCurrentState(s)
    m.initializeWorkSpace(wk)
    
    
    tau_xy = np.array([s.s0[3]/np.sqrt(2.)])
    eps_xy = np.array([s.e0[3]/np.sqrt(2.)])
    tau_yz = np.array([s.s0[5]/np.sqrt(2.)])
    eps_yz = np.array([s.e0[5]/np.sqrt(2.)])
    tau_xz = np.array([s.s0[4]/np.sqrt(2.)])
    eps_xz = np.array([s.e0[4]/np.sqrt(2.)])
    eps_xx = np.array([s.e0[0]])
    sig_xx = np.array([s.s0[0]])
    eps_yy = np.array([s.e0[1]])
    sig_yy = np.array([s.s0[1]])
    eps_zz = np.array([s.e0[2]])
    sig_zz = np.array([s.s0[2]])
    pc = np.array([-pc0])
    for i in range(len(t_discrete)-1):
        m.execute(s, wk, t_discrete[i], t_discrete[i + 1])
        eps_xy = np.append(eps_xy,s.e1[3]/np.sqrt(2.)) #Kelvin mapping backwards!
        tau_xy = np.append(tau_xy,s.s1[3]/np.sqrt(2.)) #Kelvin mapping backwards!
        eps_xz = np.append(eps_xz,s.e1[4]/np.sqrt(2.)) #Kelvin mapping backwards!
        tau_xz = np.append(tau_xz,s.s1[4]/np.sqrt(2.)) #Kelvin mapping backwards!
        eps_yz = np.append(eps_yz,s.e1[5]/np.sqrt(2.)) #Kelvin mapping backwards!
        tau_yz = np.append(tau_yz,s.s1[5]/np.sqrt(2.)) #Kelvin mapping backwards!
        sig_xx = np.append(sig_xx,s.s1[0])
        eps_xx = np.append(eps_xx,s.e1[0])
        sig_yy = np.append(sig_yy,s.s1[1])
        eps_yy = np.append(eps_yy,s.e1[1])
        sig_zz = np.append(sig_zz,s.s1[2])
        eps_zz = np.append(eps_zz,s.e1[2])
        pc = np.append(pc,s.getInternalStateVariableValue("PreConsolidationPressure"))

    return -sig_xx, -eps_xx, -sig_yy, -eps_yy, -sig_zz, -eps_zz, -tau_xy, -eps_xy, -tau_xz, -eps_xz, -tau_yz, -eps_yz, -pc